<a href="https://colab.research.google.com/github/datasci-rahul/porfolio/blob/main/Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun May 12 16:19:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }

!pip -q install torch diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==7.7.1

In [ ]:
#@title 👇 Selecting Model { form-width: "20%", display-mode: "form" }

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
from google.colab import output
output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
def get_scheduler(name):

  match name:

    case "EulerAncestralDiscreteScheduler":
      return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "DDIMScheduler":
      return DDIMScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "EulerDiscreteScheduler":
      return EulerDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "UniPCMultistepScheduler":
      return UniPCMultistepScheduler.from_pretrained(select_model.value, subfolder="scheduler")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value,
      scheduler=get_scheduler(select_sampler.value),
      torch_dtype=torch.float16,
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)


In [ ]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }

'''
**Prompt** - Description of the image
**Negative Prompt** - Things you don't want to see or ignore in the image
**Steps** - Number of denoising steps. Higher steps may lead to better results but takes longer time to generate the image. Default is `30`.
**CFG** - Guidance scale ranging from `0` to `20`. Lower values allow the AI to be more creative and less strict at following the prompt. Default is `7.5`.
**Seed** - A random value that controls image generation. The same seed and prompt produce the same images. Set `-1` for using random seed values.
'''
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=30,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print("Running...")

    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed),
    ).images
    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)

# **ModelScope**

In [ ]:
!pip install modelscope==1.4.2
!pip install open_clip_torch
!pip install pytorch-lightning

  Using cached open_clip_torch-2.24.0-py3-none-any.whl (1.5 MB)
  Using cached ftfy-6.2.0-py3-none-any.whl (54 kB)
  Using cached timm-0.9.16-py3-none-any.whl (2.2 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-m

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys

p = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis')

2024-05-12 16:21:09,878 - modelscope - INFO - PyTorch version 2.2.1+cu121 Found.
2024-05-12 16:21:09,889 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-05-12 16:21:09,890 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-05-12 16:21:09,895 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-12 16:21:10,082 - modelscope - INFO - Loading done! Current index file version is 1.4.2, with md5 f652e1480aa3d3ed036eafe4e3221492 and a total number of 842 components indexed
2024-05-12 16:21:23,132 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.0
Downloading: 100%|██████████| 782k/782k [00:00<00:00, 10.2MB/s]
Downloading: 100%|██████████| 796k/796k [00:00<00:00, 7.88MB/s]
Downloading: 100%|██████████| 737k/737k [00:00<00:00, 9.38MB/s]
Downloading: 100%|██████████| 846k/846k [00:00<00:00, 10.3MB/s]
Downloading: 100%|██████████| 729k/729

In [ ]:
test_text = {
        'text': 'Darth Vader is surfing on waves',
    }
output_video_path = p(test_text,)[OutputKeys.OUTPUT_VIDEO]
print('output_video_path:', output_video_path)

2024-05-12 16:26:48,299 - modelscope - WARNING - task text-to-video-synthesis input definition is missing
2024-05-12 16:27:47,266 - modelscope - WARNING - task text-to-video-synthesis output keys are missing


output_video_path: /tmp/tmpjr8g77j1.mp4


In [ ]:
# Download result
from google.colab import files
files.download(output_video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ZeroScope**

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers accelerate torch


  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-ash45fy9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-ash45fy9
  Resolved https://github.com/huggingface/diffusers.git to commit 98ba18ba55f1f74b3d3b51fbe1db2c6978c9f9ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.28.0.dev0-py3-none-any.whl size=2123802 sha256=4d1b9cb50f6020a68ed141617c5c8824f4d0041d32f5ab444f86e02a5cac6de2
  Stored in directory: /tmp/pip-ephem-wheel-cache-k_lch7w3/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.4 MB/s eta 0:00:00


# **zeroscope_v2 576w**

In [ ]:
!pip install accelerate

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

prompt = "Darth Vader is surfing on waves"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=24).frames
video_path = export_to_video(video_frames)

vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

# **zeroscope_v2 XL**

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

pipe.enable_vae_slicing()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1) # disable if enough memory as this slows down significantly

prompt = "Darth Vader is surfing in waves"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=36).frames
video_path = export_to_video(video_frames)

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
#!pip install opencv-python

In [ ]:
print(type(video_frames))

<class 'torch.Tensor'>


In [ ]:
video_frames = torch.from_numpy(video_frames)

In [ ]:
video_frames = video_frames.to(torch.uint8)

In [ ]:
import cv2
import torch
import numpy as np

def export_to_video(video_frames, output_video_path, fps):
    # Convert the tensor to a NumPy array
    video_frames = video_frames.numpy()

    # Convert the NumPy array to the appropriate data type
    video_frames = video_frames.astype(np.uint8)

    # Extract the height and width from the first frame
    h, w, c = video_frames[0][0].shape

    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps=fps, frameSize=(w, h))

    # Write the frames to the video
    for i in range(video_frames.shape[1]):
        frame = video_frames[0][i]
        video_writer.write(frame)

    # Close the video writer
    video_writer.release()
video_path = export_to_video(video_frames, output_video_path="output.mp4", fps=24)